In [1]:
import pandas as pd
import duckdb

In [2]:
df=pd.read_csv("/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/bangalore/raw/location_poi_data.csv")

In [3]:
all_data = df.copy()
con = duckdb.connect(database='duckdb_gmaps.db', read_only=False)
con.register('all_unique_gmaps_data', all_data)


# create a table from the dataframe so that it persists
con.execute("drop TABLE if exists gmaps_data_eye  ")

con.execute("CREATE TABLE gmaps_data_eye AS (SELECT * FROM all_unique_gmaps_data)")
#con.close()


con.execute("""PRAGMA create_fts_index(gmaps_data_eye, 'id', 'name', stemmer='porter', stopwords='english',
                 ignore='(\\.|[^a-z])+', strip_accents=1, lower=1, overwrite=1)
            """)


In [ ]:
df2=con.execute("""SELECT *,'reliance digital' as brand_name, fts_main_gmaps_data.match_bm25(
        place_id, 'Reliance Digital', fields := 'poi_name',conjunctive := 1) AS score FROM gmaps_data
    """).fetch_df()

In [13]:
b=["Gold's Gym","Kaizen Fitness","Stayfit Health & Fitness World","The little gym","Zeus Fitness","F45 Training","Cross Fit","Namma Crossfit","Snap Fitness","Fitness planet", "Planet fitness",'iconic fitness' , 'game on fitness'
]
b=['Dominos Pizza', 'naturals salon', 'Lenskart', 'toni guy', 'plum sugar','Simpli Namdrai','ratnadeep',"natures basket", 'dmart' ,'vishal mega mart','more','reliance','zudio','titan eye+','Nail Salon']

In [14]:
all_brands=[]
for i in b:
    df2=con.execute(f"""select * from(SELECT *, fts_main_gmaps_data_eye.match_bm25(
            id, '{i.replace("'",'')}', fields := 'name',conjunctive := 1) AS score FROM gmaps_data_eye) where score>0
        """).fetch_df()
    brands=df2['brand_id']
    all_brands.append(df2)
    

In [21]:
pd.concat(all_brands).query("brand_id=='poorvika'")

,id,name,lat,lng,cid,brand_id,brand_name,number_of_votes,rating,place_url,source,category,top_category,type,created_at,score
230,ChIJVwKO5iM-rjsRYg3o_5zWaR8,Poorvika Mobiles Banashankari - Near Reliance ...,12.938217,77.555702,0x3bae3e23e68e0257:0x1f69d69cffe80d62,poorvika,Poorvika,2494,4.7,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,electronic_store,retail,N_A,1695212099,2.398878


In [16]:
brands=list(set(pd.concat(all_brands)[['id','name','brand_id','brand_name','score']]['brand_id'].tolist()))
brands=list(filter(lambda x:x!='N_A',brands))
brands

['jio',
 'reliancejewels',
 "zomato_domino's_pizza",
 'ratnadeep',
 'ril',
 'dominos',
 'dmartindia',
 'reliancedigitalstores',
 'ajio',
 'naturesbasket',
 'marksandspencer',
 'reliancedigital',
 'relianceretail',
 'dmartreadyonline',
 'naturals',
 'zudio',
 'lenskart',
 'poorvika',
 'reliancesmartbazaar',
 'vishalmegamart',
 'reliancesmartpoint',
 'toniandguy',
 'dmart']